In [5]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
import time 
import re
import csv
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import math




In [6]:
driver = webdriver.Chrome()

## Using two random zip codes for now
zips = [20852, 10003]

## Make an empty list to store dictionaries later 
list_of_dicts = []

for element in zips:

    ## Navigate to the link based on the zipcodes provided in the list 'zips'
    link ='https://www.greatschools.org/search/search.zipcode?sort=rating&zip=' + str(element) 
    driver.get(link)

    ## Find the total number of pages per zip based on total number of schools 
    num_pages = driver.find_element(By.XPATH,'.//div[@class="pagination-summary"]').text
    total_schools = int(num_pages[19:21])
    num_pages = int(math.ceil(total_schools/25))

    ## For each page in each zip, loop through and pull out the relevant information
    for x in range(num_pages):
        page_num = x + 1
        if(page_num > 1):
            link = 'https://www.greatschools.org/search/search.zipcode?page=' + str(page_num) + '&sort=rating&view=list&zip='+str(element)
            driver.get(link)


        ## get a list of all of the schools on the left 
        schools = driver.find_elements(By.XPATH,'//li[@class=" unsaved"]')

        ## Loop through all schools 
        for school in schools: 
            name = school.find_element(By.XPATH,'.//a[@class="name"]').text
            address = school.find_element(By.XPATH,'.//div[@class="address"]').text

            ## Find level and enrollment -- they share the same tag, so separate them out
            ## Note, sometimes enrollment is not available, so check if it is and if not, make it na

            level_and_enrollment = school.find_elements(By.XPATH,'.//span[@class="open-sans_sb"]')
            if(len(level_and_enrollment) > 1):
                level = level_and_enrollment[0].text
                enrollment = level_and_enrollment[1].text
            else:
                level = level_and_enrollment[0].text
                enrollment = 'NA'

            rating = school.find_elements(By.XPATH,'.//span[@class="tipso_style"]')
            rating = rating[0].text[0]

            #If rating is unavaiable, will return 'C' -- change that to 'NA'
            if(rating == 'C'):
                rating = 'NA'

            scale = school.find_elements(By.XPATH,'.//div[@class="scale"]')
            scale = scale[0].text

            ## make a dictionary for each entry and add it to the list_of_dicts
            dict = {}
            dict['name'] = name
            dict['address'] = address
            dict['level'] = level
            dict['enrollment'] = enrollment
            dict['rating'] = rating
            dict['scale'] = scale
            dict['zip'] = element
            list_of_dicts.append(dict)

        # next = driver.find_elements(By.XPATH,'.//a[@class="anchor-button   anchor-button"]')
        # next[1].click() 

In [7]:
final_df = pd.DataFrame(list_of_dicts)

In [8]:
final_df

,name,address,level,enrollment,rating,scale,zip
0,Farmland Elementary,"7000 Old Gate Road, Rockville, MD, 20852","Public district, PK-5",830,9,Above average,20852
1,Tilden Middle School,"11211 Old Georgetown Road, Rockville, MD, 20852","Public district, 6-8",989,7,Above average,20852
2,Richard Montgomery High,"250 Richard Montgomery Drive, Rockville, MD, 2...","Public district, 9-12","2,483",5,Average,20852
3,Green Acres School,"11701 Danville Drive, Rockville, MD, 20852","Private, PK-8",298,NA,Currently unrated,20852
4,Georgetown Preparatory School,"10900 Rockville Pike, North Bethesda, MD, 20852","Private, 9-12",487,NA,Currently unrated,20852
...,...,...,...,...,...,...,...
63,Beginnings Nursery School,"130 East 16th Street, New York, NY, 10003","Private, PK",NA,NA,Currently unrated,10003
64,Manhattan Kid's Club,"21 East 13th Street, New York, NY, 10003","Private, PK",NA,NA,Currently unrated,10003
65,Third Street Preschool,"235 East 11th Street, New York, NY, 10003","Private, PK-4",NA,NA,Currently unrated,10003
66,Brotherhood Nursery School,"28 Gramercy Park South, New York, NY, 10003","Private, PK",NA,NA,Currently unrated,10003
